In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
try:
    import cv2
except:
    print("cv2 not found installing")
    !pip install cv2
    import cv2

In [2]:
imu_dir = r"C:\Users\ravik\practice\projects\Hand Pose detection\glovedata/"
vid_dir = r"C:\Users\ravik\practice\projects\Hand Pose detection\mediapipe videos/"
output_dir = "processed"

In [36]:
df = pd.read_csv(imu_dir+"participant_13.csv")
df.head()

,Sensors: lowerArm (0),thumb (1),index (2),mid (3),ring (4),pinky (5),palm (6)
0,Timestamp,Tick,Sensor,QuatI,QuatJ,QuatK,QuatSum
1,0.0,38586,0,-4.329637285359677e-17,-0.7071067811865476,0.7071067811865475,-4.329637285359678e-17
2,0.0,38586,1,-0.5,-0.5,0.4999999999999999,-0.5000000000000001
3,0.0,38586,2,-0.5,-0.5,0.4999999999999999,-0.5000000000000001
4,0.0,38587,3,-0.5,-0.5,0.4999999999999999,-0.5000000000000001


In [37]:
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)
df.head()

,Timestamp,Tick,Sensor,QuatI,QuatJ,QuatK,QuatSum
0,0.0,38586,0,-4.329637285359677e-17,-0.7071067811865476,0.7071067811865475,-4.329637285359678e-17
1,0.0,38586,1,-0.5,-0.5,0.4999999999999999,-0.5000000000000001
2,0.0,38586,2,-0.5,-0.5,0.4999999999999999,-0.5000000000000001
3,0.0,38587,3,-0.5,-0.5,0.4999999999999999,-0.5000000000000001
4,0.001,38587,4,-0.5,-0.5,0.4999999999999999,-0.5000000000000001


In [38]:
preserved_columns = df[['Timestamp',' Sensor']]

In [39]:
# Normalizing quaternions while preserving relationships
def normalize_quaternions(data):
    normalized_quaternions = []
    for _, row in data.iterrows():
        quat = np.array([row[' QuatI'], row[' QuatJ'], row[' QuatK'], row[' QuatSum']])
        norm = np.linalg.norm(quat)  # Calculate magnitude
        normalized_quat = quat / norm # Rescale to unit norm
        normalized_quaternions.append(normalized_quat)
    return pd.DataFrame(normalized_quaternions, columns=['QuatI', 'QuatJ', 'QuatK', 'QuatSum'])


In [40]:
quaternion_cols = [' QuatI',' QuatJ',' QuatK',' QuatSum']

for col in quaternion_cols:
    df[col] = pd.to_numeric(df[col])
    
normalized_quaternions = normalize_quaternions(df)
normalized_quaternions.head()

,QuatI,QuatJ,QuatK,QuatSum
0,-4.329637e-17,-0.707107,0.707107,-4.329637e-17
1,-5.000000e-01,-0.500000,0.500000,-5.000000e-01
2,-5.000000e-01,-0.500000,0.500000,-5.000000e-01
3,-5.000000e-01,-0.500000,0.500000,-5.000000e-01
4,-5.000000e-01,-0.500000,0.500000,-5.000000e-01


In [41]:
#this is to check if the rescaling is done properly and the relation between quaternions is preserved
data = normalized_quaternions.iloc[10]
print(1 == ((data['QuatI']**2 + data['QuatJ']**2 + data['QuatK']**2 + data['QuatSum']**2)**0.5))

True


In [42]:
final_data = pd.concat([preserved_columns, normalized_quaternions], axis=1)
final_data['Timestamp'] = pd.to_numeric(final_data['Timestamp'])
final_data.head()


,Timestamp,Sensor,QuatI,QuatJ,QuatK,QuatSum
0,0.000,0,-4.329637e-17,-0.707107,0.707107,-4.329637e-17
1,0.000,1,-5.000000e-01,-0.500000,0.500000,-5.000000e-01
2,0.000,2,-5.000000e-01,-0.500000,0.500000,-5.000000e-01
3,0.000,3,-5.000000e-01,-0.500000,0.500000,-5.000000e-01
4,0.001,4,-5.000000e-01,-0.500000,0.500000,-5.000000e-01


In [43]:
final_data.to_csv(output_dir+"/imu/participant_13.csv", index=False)

In [125]:
timestamps = final_data['Timestamp'].drop_duplicates()
timestamps.head()

0    0.000
3    0.035
5    0.039
6    0.040
7    0.043
Name: Timestamp, dtype: float64

In [126]:
def extract_frames(video_path, timestamps, output_dir):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open the file specified")
        return
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Video FPS: {fps}")

    for i, timestamp in enumerate(timestamps):
        frame_no = int(timestamp * fps)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)

        ret, frame = cap.read()
        if ret:
            frame_path = f"{output_dir}/frames/participant_10/frame_02_{timestamp}.jpg" #change the participant number manually
            cv2.imwrite(frame_path, frame)
        else:
            print(f"ERROR: unable to read frame at timestamp {timestamp}")

    cap.release()
    

In [127]:
extract_frames(r"C:\Users\ravik\practice\projects\Hand Pose detection\mediapipe videos\OBS recordings\participant_10.mkv", timestamps, output_dir)

Video FPS: 30.0
ERROR: unable to read frame at timestamp 210.456
ERROR: unable to read frame at timestamp 210.457
ERROR: unable to read frame at timestamp 210.521
ERROR: unable to read frame at timestamp 210.522
ERROR: unable to read frame at timestamp 210.523
ERROR: unable to read frame at timestamp 210.524
ERROR: unable to read frame at timestamp 210.525
ERROR: unable to read frame at timestamp 210.535
ERROR: unable to read frame at timestamp 210.537
ERROR: unable to read frame at timestamp 210.539
ERROR: unable to read frame at timestamp 210.585
ERROR: unable to read frame at timestamp 210.588
ERROR: unable to read frame at timestamp 210.603
ERROR: unable to read frame at timestamp 210.604
ERROR: unable to read frame at timestamp 210.61
ERROR: unable to read frame at timestamp 210.612
ERROR: unable to read frame at timestamp 210.613
ERROR: unable to read frame at timestamp 210.614
ERROR: unable to read frame at timestamp 210.615
ERROR: unable to read frame at timestamp 210.621
ERROR

In [53]:
grouped_data = final_data.groupby(' Sensor')
for name, group in grouped_data:
    print(f"Group: {name}")
    print(group.head())
    print("-" * 30)

Group: 0
    Timestamp  Sensor     QuatI     QuatJ     QuatK   QuatSum
0       0.000       0  0.135776 -0.690791  0.703221  0.099264
6       0.035       0  0.135819 -0.690790  0.703219  0.099221
12      0.073       0  0.135819 -0.690790  0.703219  0.099221
18      0.107       0  0.135863 -0.690789  0.703218  0.099178
24      0.135       0  0.135992 -0.690742  0.703258  0.099048
------------------------------
Group: 1
    Timestamp  Sensor     QuatI     QuatJ     QuatK   QuatSum
1       0.001       1 -0.561731 -0.481653 -0.120886 -0.661706
7       0.035       1 -0.561687 -0.481670 -0.120930 -0.661723
13      0.073       1 -0.561687 -0.481670 -0.120930 -0.661723
19      0.107       1 -0.561687 -0.481670 -0.120930 -0.661723
25      0.135       1 -0.561687 -0.481670 -0.120930 -0.661723
------------------------------
Group: 2
    Timestamp  Sensor     QuatI     QuatJ     QuatK   QuatSum
2       0.001       2 -0.700921  0.431647  0.088556 -0.560846
8       0.035       2 -0.700925  0.431583  

In [60]:
def calculate_features(group):
    features = pd.DataFrame()
    #preserving original quaternions and timestamp
    features['Timestamp'] = group['Timestamp']
    features['Sensor'] = group[' Sensor']
    features['QuatI'] = group['QuatI']
    features['QuatJ'] = group['QuatJ']
    features['QuatK'] = group['QuatK']
    features['QuatSum'] = group['QuatSum']

    #angle of rotation
    features['Rotation_Angle'] = 2 * np.arccos(group['QuatSum'])
    
    #axis of rotation
    features['Axis_X'] = group['QuatI'] / np.sqrt(group['QuatI']**2 + group['QuatJ']**2 + group['QuatK']**2)
    features['Axis_Y'] = group['QuatJ'] / np.sqrt(group['QuatI']**2 + group['QuatJ']**2 + group['QuatK']**2)
    features['Axis_Z'] = group['QuatK'] / np.sqrt(group['QuatI']**2 + group['QuatJ']**2 + group['QuatK']**2)

    #angluar velocity
    time_deltas = group['Timestamp'].diff()  # Time difference in seconds
    rotation_angle_deltas = 2 * np.arccos(group['QuatSum']).diff()  # Change in rotation angle
    features['Angular_Velocity'] = rotation_angle_deltas / time_deltas # Angular velocity (angle/time)
    features['Angular_Velocity'] = features['Angular_Velocity'].fillna(0) #for first row filling it zero

    return features


In [61]:
features_by_sensor = grouped_data.apply(calculate_features)
features_by_sensor.reset_index(drop=True, inplace=True)

C:\Users\ravik\AppData\Local\Temp\ipykernel_25508\1252205206.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features_by_sensor = grouped_data.apply(calculate_features)


In [64]:
features_by_sensor.head()

,Timestamp,Sensor,QuatI,QuatJ,QuatK,QuatSum,Rotation_Angle,Axis_X,Axis_Y,Axis_Z,Angular_Velocity
0,0.000,0,0.135776,-0.690791,0.703221,0.099264,2.942737,0.136450,-0.694220,0.706711,0.000000
1,0.035,0,0.135819,-0.690790,0.703219,0.099221,2.942823,0.136493,-0.694216,0.706707,0.002478
2,0.073,0,0.135819,-0.690790,0.703219,0.099221,2.942823,0.136493,-0.694216,0.706707,0.000000
3,0.107,0,0.135863,-0.690789,0.703218,0.099178,2.942910,0.136536,-0.694211,0.706703,0.002551
4,0.135,0,0.135992,-0.690742,0.703258,0.099048,2.943170,0.136664,-0.694155,0.706733,0.009294
